## 3. ML

In [4]:
# загрузим библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn import linear_model 
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split 
%matplotlib inline
plt.style.use('seaborn')

C:\Users\tani_\AppData\Local\Temp\ipykernel_4164\1250253059.py:24: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [5]:
# загрузим очищенные и нормализованные данные
norm_df = pd.read_csv('data/norm_df.csv')
norm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165481 entries, 0 to 165480
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   baths                     165481 non-null  float64
 1   fireplace                 165481 non-null  int64  
 2   city                      165481 non-null  int64  
 3   sqft                      165481 non-null  float64
 4   zipcode                   165481 non-null  int64  
 5   state                     165481 non-null  int64  
 6   target                    165481 non-null  float64
 7   pool                      165481 non-null  int64  
 8   school_max_rating         165481 non-null  float64
 9   shcool_mean_distance      165481 non-null  float64
 10  Year built                165481 non-null  int64  
 11  Remodeled                 165481 non-null  int64  
 12  Heating                   165481 non-null  int64  
 13  Cooling                   165481 non-null  i

Разделим набор данных на матрицу наблюдений X (таблица из объектов и их признаков) и столбец правильных ответов y

In [6]:
# X - матрица наблюдений, y - столбец правильных ответов
X = norm_df.drop('target', axis=1)
y = norm_df['target']

In [7]:
X

,baths,fireplace,city,sqft,zipcode,state,pool,school_max_rating,shcool_mean_distance,Year built,...,propertyType_apartment,propertyType_condo,propertyType_family_home,propertyType_historical,propertyType_land,propertyType_mobile_home,propertyType_modern,propertyType_other,propertyType_ranch,propertyType_townhouse
0,0.5,1,1229,1.255020,778,19,0,0.333333,1.764706,27,...,False,False,True,False,False,False,False,False,False,False
1,0.5,0,1236,0.298193,3760,33,0,-0.666667,-0.470588,21,...,False,False,True,False,False,False,False,False,False,False
2,-1.0,0,1018,-0.143574,1323,5,0,-1.000000,0.058824,24,...,False,False,False,False,False,False,False,True,False,False
3,0.0,0,781,1.945783,1937,8,0,0.333333,0.941176,11,...,False,False,True,False,False,False,False,False,False,False
4,-1.0,0,591,1.485944,2418,29,0,0.000000,1.176471,24,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165476,0.5,1,708,1.026104,3007,21,1,-0.333333,-0.294118,28,...,False,False,True,False,False,False,False,False,False,False
165477,0.5,0,128,0.373494,1273,5,1,0.666667,0.000000,24,...,False,False,True,False,False,False,False,False,False,False
165478,0.0,0,1283,0.258032,1143,5,0,0.000000,-0.294118,24,...,False,False,True,False,False,False,False,False,False,False
165479,0.5,1,591,0.142570,2430,29,0,-1.000000,-0.588235,24,...,False,False,True,False,False,False,False,False,False,False


Разделяем всю выборку на тренировочную и тестовую в соотношении 80/20.

На тренировочной выборке будем обучать модели, а на тестовой проверять их качество. 

In [8]:
#Создаем тренировочную и тестовую выборки с помощью train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, #таблица с наблюдениями и столбец с правильным ответами
    test_size=0.2, #размер тестовой выборки
    random_state=42 #число отвечающее за генерацию случайных чисел
)
#Выводим размеры полученных выборок
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))

Train shape: (132384, 34)
Test shape: (33097, 34)


### Моделирование и оценка модели

Cоздадим функцию, которая будет расчитывать метрики:

1. $R^2$ - коэффициент детерминации 
2. MAE - средняя абсолютная ошибка (целевая метрика)
3. MSE - средняя квадратическая ошибка

In [9]:
# функция для расчета метрик
def print_metrics(y_train, y_train_predict, y_test, y_test_predict):
    print('Train R^2: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))
    print('Train MAE: {:.3f}'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
    print('Train MSE: {:.3f}'.format(metrics.mean_squared_error(y_train, y_train_predict)))
    print('\n')
    print('Test R^2: {:.3f}'.format(metrics.r2_score(y_test, y_test_predict)))
    print('Test MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_test_predict)))
    print('Test MSE: {:.3f}'.format(metrics.mean_squared_error(y_test, y_test_predict)))

### 1. Модель линейной регрессии

In [10]:
# Инициализируем объект класса линейная регрессия
lr = linear_model.LinearRegression()
# Обучаем модель - ищем параметры
lr.fit(X_train, y_train)
# Делаем предсказание для каждой из выборок
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)
# Выводим результирующие метрики
print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.301
Train MAE: 0.443
Train MSE: 0.305


Test R^2: 0.298
Test MAE: 0.443
Test MSE: 0.305


Выводы:

1. Коэффициент детерминации R^2 для обучающего набора составляет 0.301, что означает, что примерно 30.1% изменчивости целевой переменной может быть объяснено моделью. Аналогично, коэффициент детерминации R^2 для тестового набора составляет 0.298, что означает, что примерно 29.8% изменчивости целевой переменной может быть объяснено моделью. Оба значения R^2 находятся в довольно низком диапазоне, что может указывать на то, что модель не очень хорошо объясняет изменчивость данных.

2. Средняя абсолютная ошибка (MAE) для обучающего набора составляет 0.443, для тестового набора также 0.443. Значения MAE показывают, что модель в среднем оказывается ошибочной на 0.443  единицы.

3. Среднеквадратическая ошибка (MSE) для обучающего и тестового набора составляет 0.305. Значения MSE показывают, насколько сильно отличаются предсказанные значения от фактических значений. Более высокое значение MSE указывает на большую дисперсию ошибок.

На основе данных метрик можно заключить, что модель машинного обучения линейной регрессии имеет относительно низкую точность предсказаний, так как значения R^2, MAE и MSE оказались довольно низкими. Для улучшения модели требуется использовать более сложные модели.

### 2. Регуляризованная линейная регрессия

In [11]:
# Создадим регрессионную модель с параметром регуляризации alpha
alpha = 0.5
ridge = Ridge(alpha=alpha)
# Обучаем модель
ridge.fit(X_train, y_train)
# Делаем предсказание для каждой из выборок
y_train_pred = ridge.predict(X_train)
y_test_pred = ridge.predict(X_test)
# Выводим результирующие метрики
print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.301
Train MAE: 0.443
Train MSE: 0.305


Test R^2: 0.298
Test MAE: 0.443
Test MSE: 0.305


Выводы:

Метрики модели регуляризованной линейной регрессии идентичны показателям метрик модели линейной регрессии.

### 3. Решающие деревья

In [12]:
trees_df = norm_df.copy()

In [13]:
# X - матрица наблюдений, y - столбец правильных ответов
X = trees_df.drop('target', axis=1)
y = trees_df['target']

In [14]:
#Создаем тренировочную и тестовую выборки с помощью train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, #таблица с наблюдениями и столбец с правильным ответами
    test_size=0.2, #размер тестовой выборки
    random_state=42 #число отвечающее за генерацию случайных чисел
)
#Выводим размеры полученных выборок
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))

Train shape: (132384, 34)
Test shape: (33097, 34)


In [15]:
# Создание и обучение модели решающего дерева
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
# Предсказание значений на тренировочной выборке
train_pred = model.predict(X_train)
# Предсказание значений на тестовой выборке
test_pred = model.predict(X_test)
# Оценка точности модели
train_accuracy = r2_score(y_train, train_pred)
test_accuracy = r2_score(y_test, test_pred)
# Вывод точности модели
# print('Train Accuracy:', train_accuracy)
# print('Test Accuracy:', test_accuracy)
# # Выводим результирующие метрики
print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.301
Train MAE: 0.443
Train MSE: 0.305


Test R^2: 0.298
Test MAE: 0.443
Test MSE: 0.305


Выводы:

1. Train R^2 (коэффициент детерминации) равен 0.301 для обучающей выборки, что означает, что примерно 30.1% дисперсии зависимой переменной объясняется моделью. Это говорит о том, что модель не очень хорошо соответствует обучающим данным, так как R^2 близок к 0. Это может указывать на недообучение модели. Test R^2 (коэффициент детерминации на тестовой выборке) также равен 0.298. Это говорит о том, что модель не очень хорошо обобщает данные на новых данных, так как R^2 также близок к 0.

2. Train MAE (средняя абсолютная ошибка) составляет 0.443. Это означает, что модель в среднем ошибается на 0.443 единицы при прогнозировании значений на обучающей выборке. Test MAE (средняя абсолютная ошибка на тестовой выборке) также составляет 0.443. Это показывает, что модель имеет схожую среднюю ошибку на тестовой выборке, как и на обучающей.

3. Train MSE (среднеквадратичная ошибка) равен 0.305. Эта метрика показывает, насколько сильно модель отклоняется от реальных значений на обучающей выборке. В данном случае, MSE равен 0.305. Test MSE (среднеквадратичная ошибка на тестовой выборке) также равен 0.305. Это говорит о том, что модель сохраняет схожее качество прогнозов на тестовой выборке, как и на обучающей.

Исходя из предоставленных метрик, можно сделать вывод, что модель Решающего леса не показывает выдающегося качества прогнозирования. Она имеет невысокий коэффициент детерминации (R^2) как на обучающей, так и на тестовой выборках, что может указывать на недообучение или несостоятельность модели. Средние абсолютные и среднеквадратичные ошибки на обучающей и тестовой выборках примерно одинаковы, что также может указывать на отсутствие переобучения, но модель требует доработки, чтобы улучшить качество прогнозов.

### 4. Случайный лес

In [16]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=42)
# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)
# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)
# Выводим результирующие метрики
print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.964
Train MAE: 0.089
Train MSE: 0.016


Test R^2: 0.750
Test MAE: 0.234
Test MSE: 0.109


Выводы:

1. Train R^2 (коэффициент детерминации) равен 0.964, что означает, что 96.4% изменчивости зависимой переменной (целевой переменной) в тренировочных данных объясняется моделью. Это высокий показатель и говорит о хорошем соответствии модели данным тренировки. Test R^2 равен 0.750, что означает, что модель объясняет около 75% изменчивости в тестовых данных. Это может быть интерпретировано как хороший уровень соответствия модели данным тестирования.

2. Train MAE (средняя абсолютная ошибка) равна 0.089. MAE показывает среднюю абсолютную разницу между предсказанными и фактическими значениями в тренировочных данных. Низкое значение MAE свидетельствует о том, что модель достаточно точна в предсказании тренировочных данных. Test MAE равна 0.234, что указывает на среднюю абсолютную разницу между предсказанными и фактическими значениями в тестовых данных. Более низкое значение MAE будет свидетельствовать о более точной модели.

3. Train MSE (среднеквадратическая ошибка) равна 0.016. MSE показывает среднюю квадратическую разницу между предсказанными и фактическими значениями в тренировочных данных. Низкое значение MSE также указывает на хорошую точность модели в тренировочных данных. Test MSE равна 0.109, показывая среднюю квадратическую разницу между предсказанными и фактическими значениями в тестовых данных. Более низкое значение MSE также будет указывать на более точную модель.

Исходя из этих метрик, можно сказать, что модель Random Forest имеет хорошую способность объяснять вариацию данных как на тренировочные, так и на тестовые данные. Однако, разница между показателями тренировочных и тестовых данных может подразумевать некоторое переобучение модели. Чтобы сделать окончательные выводы и принять решение об использовании этой модели в реальных условиях, проведем дополнительный анализ и сравнение с другими моделями.

### 5. Гауссов процесс (Из-за необходимого большое объема памяти не удлалось воспроизвести эту модель)

In [17]:
# # Создание объекта модели Гауссового процесса с выбором ядра (RBF - радиально-базисная функция)
# kernel = 1.0 * RBF(length_scale=1.0)
# # Инициализация модели Гауссового процесса
# gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
# # Обучение модели на тренировочных данных
# gp.fit(X_train, y_train)
# # Предсказания на обучающих и тестовых данных
# y_train_pred = gp.predict(X_train)
# y_test_pred = gp.predict(X_test)
# # Выводим результирующие метрики
# print_metrics(y_train, y_train_pred, y_test, y_test_pred)

### 6. Эластичная модель

In [18]:
# Создание модели
el_model = ElasticNetCV(cv=6, random_state=42)
# Обучение модели на тренировочных данных
el_model.fit(X_train, y_train)
# Предсказания на обучающих и тестовых данных
y_train_pred = el_model.predict(X_train)
y_test_pred = el_model.predict(X_test)
# Выводим результирующие метрики
print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.203
Train MAE: 0.484
Train MSE: 0.348


Test R^2: 0.200
Test MAE: 0.483
Test MSE: 0.348


Выводы:

1. R^2 (коэффициент детерминации) для обучающей и тестовой выборок составляет около 0.20. Это означает, что модель объясняет всего лишь 20% дисперсии зависимой переменной. Низкое значение R^2 может свидетельствовать о том, что модель плохо соответствует данным.

2. MAE (средняя абсолютная ошибка) для обучающей и тестовой выборок составляет около 0.484 и 0.483 соответственно. MAE измеряет среднее абсолютное отклонение прогнозов модели от фактических значений. Значения MAE довольно близки друг к другу и находятся на относительно низком уровне, что может быть приемлемо в зависимости от конкретной задачи.

3. MSE (средняя квадратичная ошибка) для обучающей и тестовой выборок также составляет около 0.348. MSE измеряет среднеквадратичное отклонение прогнозов модели от фактических значений. Значения MSE также сходятся между обучающей и тестовой выборками и находятся на относительно низком уровне.

В целом, метрики модели показывают, что модель имеет низкую способность объяснения дисперсии зависимой переменной (низкий R^2), но при этом она делает прогнозы с небольшой средней абсолютной и среднеквадратичной ошибкой. 

### 7. Модель полиномиальной регрессии

Попробуем добавить в данные полиномиальные признаки и построить модель полиномиальной регрессии.

In [19]:
pol_df = norm_df.copy()

In [20]:
# X - матрица наблюдений, y - столбец правильных ответов
X = pol_df.drop('target', axis=1)
y = pol_df['target']

In [21]:
#Создаем тренировочную и тестовую выборки с помощью train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, #таблица с наблюдениями и столбец с правильным ответами
    test_size=0.2, #размер тестовой выборки
    random_state=42 #число отвечающее за генерацию случайных чисел
)
#Выводим размеры полученных выборок
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))

Train shape: (132384, 34)
Test shape: (33097, 34)


In [22]:
# Создаем объект PolynomialFeatures для задания степени полинома
degree = 2  # Задайте желаемую степень полинома
poly_features = PolynomialFeatures(degree=degree)
# Преобразуем данные в полиномиальные признаки
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)
# Создаем модель линейной регрессии
model = LinearRegression()
# Обучаем модель на тренировочных данных
model.fit(X_train_poly, y_train)
# Предсказываем значения на тренировочной и тестовой выборках
y_train_pred = model.predict(X_train_poly)
y_test_pred = model.predict(X_test_poly)
# Оценка качества модели
print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train R^2: 0.439
Train MAE: 0.391
Train MSE: 0.245


Test R^2: 0.426
Test MAE: 0.391
Test MSE: 0.249


Выводы:
    
1. Train R^2 (коэффициент детерминации) равен 0.439, что означает, что модель объясняет примерно 43.9% дисперсии в данных обучающей выборки. Это может свидетельствовать о том, что модель хорошо соответствует обучающим данным, но также может быть подвержена переобучению, если значение R^2 слишком близко к 1.

2. Train MAE (средняя абсолютная ошибка) равен 0.391. Это указывает на среднее абсолютное отклонение модели от фактических значений в обучающей выборке. Низкое значение MAE говорит о том, что модель хорошо приближает данные в обучающей выборке.

3. Train MSE (среднеквадратическая ошибка) равен 0.245. MSE измеряет среднее квадратичное отклонение модели от фактических данных. Здесь также низкое значение MSE указывает на хорошее приближение модели к данным обучающей выборки.

Test R^2, Test MAE и Test MSE имеют схожие значения с соответствующими метриками на обучающей выборке. Это может указывать на то, что модель обобщает хорошо на новых данных (тестовой выборке) и не переобучена.

In [23]:
metrics = {'Metrics': ['Train R^2', 'Train MAE', 'Train MSE', 'Test R^2', 'Test MAE', 'Test MSE'],
        'Linear Regression': [0.301, 0.443, 0.305, 0.298, 0.443, 0.305],
        'Linear Regression with Ridge': [0.301, 0.443, 0.305, 0.298, 0.443, 0.305],
        'Decision Tree Regressor': [0.301, 0.443, 0.305, 0.298, 0.443, 0.305],
        'Random Forest Regressor': [0.964, 0.089, 0.016, 0.750, 0.234, 0.109],
        'Elastic Net CV': [0.203, 0.484, 0.348, 0.200, 0.483, 0.348],
        'Polynomial Features':[0.439, 0.391, 0.245, 0.426, 0.391, 0.249]}

df_metric = pd.DataFrame(metrics)
df_metric.head(6)

,Metrics,Linear Regression,Linear Regression with Ridge,Decision Tree Regressor,Random Forest Regressor,Elastic Net CV,Polynomial Features
0,Train R^2,0.301,0.301,0.301,0.964,0.203,0.439
1,Train MAE,0.443,0.443,0.443,0.089,0.484,0.391
2,Train MSE,0.305,0.305,0.305,0.016,0.348,0.245
3,Test R^2,0.298,0.298,0.298,0.750,0.200,0.426
4,Test MAE,0.443,0.443,0.443,0.234,0.483,0.391
5,Test MSE,0.305,0.305,0.305,0.109,0.348,0.249


Общий вывод:

1. R-квадрат (R^2):

Модель "Random Forest Regressor" показывает наилучший результат на тестовой выборке с R^2 равным 0.750. Это говорит о том, что эта модель лучше всего объясняет вариацию в данных среди всех рассмотренных моделей.
Линейная регрессия, линейная регрессия с регуляризацией Ridge и Elastic Net CV показывают схожие значения R^2 как на обучающей, так и на тестовой выборках (около 0.30), что говорит о их относительной слабости по сравнению с "Random Forest Regressor".

2. Средняя абсолютная ошибка (MAE):

Модель "Random Forest Regressor" также имеет наименьшее значение MAE как на обучающей, так и на тестовой выборке, что указывает на то, что она предсказывает целевую переменную с наименьшей средней ошибкой.
Линейная регрессия с регуляризацией Ridge и линейная регрессия также имеют схожие значения MAE как на обучающей, так и на тестовой выборке.

3. Среднеквадратическая ошибка (MSE):

По MSE модель "Random Forest Regressor" также показывает лучший результат как на обучающей, так и на тестовой выборке.
Линейная регрессия, линейная регрессия с регуляризацией Ridge и Elastic Net CV имеют близкие значения MSE как на обучающей, так и на тестовой выборке.

Модель "Random Forest Regressor" является наилучшей среди рассмотренных моделей, так как она имеет наилучшие показатели R^2, MAE и MSE как на обучающей, так и на тестовой выборке. Однако, необходимо помнить о возможной переобученности данной модели, так как разница между показателями на обучающей и тестовой выборке довольно большая.

In [28]:
import pickle

# Сохранение модели 'RandomForestRegressor' в файл pickle
with open('diploma_project/application/model/working_model.pkl', 'wb') as f:
    pickle.dump(rf_regressor, f)